In [6]:
import json
import pandas as pd
from pathlib import Path
from typing import NamedTuple, Tuple, List


In [3]:
res_patgh = Path("./Results").resolve()

methods = ["original", "stage_1", "stage_2"]

In [8]:
class Results(NamedTuple):
    reported_cost: float
    lines: pd.DataFrame
    assignments: pd.DataFrame
    flows: pd.DataFrame
    mt_cost: float
    mod_cost: float
    mod_cost_total: float
    dropped_requests_count: int

def get_results(path: Path) -> Results:
    main_result = json.load((path / "metrics.json").open())
    reported_cost = main_result["objective_value"]
    lines = pd.read_csv(path / "used_lines.csv")
    assignments = pd.read_csv(path / "passenger_assignments.csv")
    flows = pd.read_csv(path / "flows.csv")
    mt_cost = lines["line_cost"].sum()
    mod_cost = assignments["mod_cost"].sum()
    mod_cost_total = (flows['cost'] * flows['flow']).sum()
    return Results(
        reported_cost=reported_cost,
        lines=lines,
        assignments=assignments,
        flows=flows,
        mt_cost=mt_cost,
        mod_cost=mod_cost,
        mod_cost_total=mod_cost_total,
        dropped_requests_count=len(assignments[assignments['line'] == 'Dropped'])
    )

def load_results_all_methods(base_path: Path, methods: List[str]) -> Tuple[dict, pd.DataFrame]:
    loaded_results = {}
    for method in methods:
        loaded_results[method] = get_results(base_path / method)

    methods_selected_results = {}
    for method, results in loaded_results.items():
        methods_selected_results[method] = {
            ("Cost", "MT"): results.mt_cost,
            ("Cost", "MoD occupied"): results.mod_cost,
            ("Cost", "Total excluding empty MoD"): results.mod_cost + results.mt_cost,
            ("Cost", "MoD empty"): results.mod_cost_total - results.mod_cost,
            ("Cost", "MoD total"): results.mod_cost_total,
            ("Cost", "Total"): results.mod_cost_total + results.mt_cost,
            ("Cost", "Reported"): results.reported_cost,
            ("Requests", "Total"): len(results.assignments),
            ("Requests", "Dropped"): results.dropped_requests_count
        }
    # create a dataframe from the dictionary
    results_df = pd.DataFrame.from_dict(methods_selected_results, orient='index')

    return loaded_results, results_df

# 100% Demand

In [ ]:
full_exp_path = res_patgh / "100_percent/budget_10000"
original_path = full_exp_path / 'original'
stage_1_path = full_exp_path / 'stage_1'
stage_2_path = full_exp_path / 'stage_2'


original_results = get_results(original_path)
stage_1_results = get_results(stage_1_path)
stage_2_results = get_results(stage_2_path)


# 10% Demand, budget 200_000

In [9]:
results_10_percent_budget_200_000, results_df_10_percent_budget_200_000 = load_results_all_methods(
    res_patgh / "10_percent/budget_200000", ["original", "stage_1"]
)
results_df_10_percent_budget_200_000

Cost                                                             \
               MT MoD occupied Total excluding empty MoD MoD empty MoD total   
original  83115.0     135498.0                  218613.0   49565.0  185063.0   
stage_1   51458.0     148536.0                  199994.0   51869.0  200405.0   

                             Requests          
             Total  Reported    Total Dropped  
original  268178.0  241777.0     1300       0  
stage_1   251863.0    1300.0     1300       0

# 1% Demand, budget 10000

In [ ]:
results_1_percent_budget_10000, results_df_1_percent_budget_10000 = load_results_all_methods(
    res_patgh / "1_percent/budget_10000"
)
results_df_1_percent_budget_10000


In [37]:
original_assignments = loaded_results['original'].assignments

# 1% Demand, budget 30000

In [95]:
results_1_percent_budget_30000, results_df_1_percent_budget_30000 = load_results_all_methods(
    res_patgh / "1_percent/budget_30000", methods
)
results_df_1_percent_budget_30000

Cost                                                             \
              MT MoD occupied Total excluding empty MoD MoD empty MoD total   
original  9847.0      23601.0                   33448.0   12618.0   36219.0   
stage_1   5083.0      24913.0                   29996.0   11678.0   36591.0   
stage_2   2461.0      20226.0                   22687.0    7065.0   27291.0   

                           Requests          
            Total Reported    Total Dropped  
original  46066.0  11568.0      130       0  
stage_1   41674.0    130.0      130       0  
stage_2   29752.0    111.0      130      19

In [86]:
results_1_percent_budget_30000_original = results_1_percent_budget_30000['original']
results_1_percent_budget_30000_stage_1 = results_1_percent_budget_30000['stage_1']
results_1_percent_budget_30000_stage_2 = results_1_percent_budget_30000['stage_2']
results_1_percent_budget_30000['stage_1'].flows['flow']

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
394    1.0
395    1.0
396    1.0
397    1.0
398    1.0
Name: flow, Length: 399, dtype: float64

In [87]:
# results_1_percent_budget_30000_original.assignments[results_1_percent_budget_30000_original.assignments['line'] == 'no_MT']
# 42 + (130-42) * 2
results_1_percent_budget_30000_stage_2.assignments[results_1_percent_budget_30000_stage_2.assignments['line'] == 'Dropped']

,passenger,line,mod_cost
0,0,Dropped,0.0
2,2,Dropped,0.0
4,4,Dropped,0.0
15,15,Dropped,0.0
24,24,Dropped,0.0
33,33,Dropped,0.0
39,39,Dropped,0.0
52,52,Dropped,0.0
53,53,Dropped,0.0
57,57,Dropped,0.0


In [93]:
results_1_percent_budget_30000_stage_2.flows['flow'].sum()

np.float64(308.0)